In [1]:
# import json
# import getpass
# import re
# from google import genai
# from google.genai import types

# def chatbot():
#     api_key = getpass.getpass('Enter your Gemini API Key: ')
#     client = genai.Client(api_key=api_key)

#     model_name = 'gemini-2.0-flash-lite'

#     config = types.GenerateContentConfig(
#         max_output_tokens=500,
#         temperature=0.1
#     )

#     extraction_config = types.GenerateContentConfig(
#         max_output_tokens=200,
#         temperature=0.0
#     )

#     print("Bot: Can I get your username and email?")

#     username = None
#     email = None

#     while True:
#         user_input = input("User: ").strip()
#         if not user_input:
#             continue

#         extraction_prompt = (
#             "Extract username and email from this text. Respond ONLY with valid JSON.\n"
#             "Use exactly these keys: 'username' and 'email' (null if missing).\n"
#             "Example: {'username': 'ssarthak', 'email': null}\n"
#             f"Text: {user_input}"
#         )

#         extraction_resp = client.models.generate_content(
#             model=model_name,
#             contents=[extraction_prompt],
#             config=extraction_config
#         )

#         print(extraction_resp.text)

#         extracted = {"username": None, "email": None}
#         try:
#             extracted = json.loads(extraction_resp.text.strip())
#         except json.JSONDecodeError:
#             try:
#                 json_str = re.search(r'{.*}', extraction_resp.text, re.DOTALL)
#                 if json_str:
#                     extracted = json.loads(json_str.group())
#             except:
#                 pass

#         username = username or extracted.get("username")
#         email = email or extracted.get("email")

#         extraction1_resp = client.models.generate_content(
#             model=model_name,
#             contents=[user_input],
#             config=extraction_config
#         )

#         if username and email:
#             print(f"Bot: Thank you {username}! We'll contact you at {email}.")
#             print(extraction1_resp.text)
#             break

#         missing = []
#         if not username: missing.append("username")
#         if not email: missing.append("email")

#         if len(missing) == 2:
#             prompt = "Can I get your username and email?"
#         elif len(missing) == 1:
#             prompt = f"Can I get your {missing[0]}?"
#         else:
#             prompt = "Is there anything else I can help with?"

#         print(f"Bot: {prompt}")

#     return username, email

# if __name__ == "__main__":
#     user, em = chatbot()
#     print(f"Stored Username: {user}, Email: {em}")


In [ ]:
import json
import getpass
import re
import time
from datetime import datetime, timedelta
from google import genai
from google.genai import types

def extract_alarm_time(client, model_name, user_input, known):
    """
    Extracts hour, minute, and period from user_input,
    focusing only on any fields that are still None in `known`.
    Uses in-context system instructions to handle both English and Nepali.
    """
    system_instructions = (
        """
You are a precise alarm-time extractor. You must read user text in English or Nepali and output ONLY valid JSON with keys 'hour', 'minute', 'period'.
- Extract numeric 'hour' (1–12) and 'minute' (0–59) when mentioned; if minute is missing but hour is given, default minute to 0.
- Determine 'period' based on any word meaning morning (AM) or evening (PM), in any language context.
- Honor any provided 'known' values and extract only the missing ones.
- If period is ambiguous or absent, set it to null.
- Respond with NOTHING except the JSON object.
Example output: {"hour":7,"minute":30,"period":"AM"}
        """
    )


    known_parts = []
    for key in ("hour", "minute", "period"):
        val = known.get(key)
        if val is not None:
            known_parts.append(f"{key}={val!r}")
    known_str = (
        "We already have: " + ", ".join(known_parts) + ".\n" if known_parts else ""
    )


    prompt = (
        system_instructions
        + "\n"
        + known_str
        + f"Text: {user_input}\n"
    )

    config = types.GenerateContentConfig(max_output_tokens=100, temperature=0.0)
    response = client.models.generate_content(
        model=model_name,
        contents=[prompt],
        config=config
    )
    print(f"AI Json response Backend {response.text.strip()}")

    try:
        return json.loads(response.text.strip())
    except json.JSONDecodeError:
        match = re.search(r'{.*}', response.text, re.DOTALL)
        if match:
            return json.loads(match.group())
        return {}


def chatbot_alarm():
    api_key = getpass.getpass("Enter your Gemini API Key: ")
    client = genai.Client(api_key=api_key)
    model_name = 'gemini-2.0-flash-lite'

    print("Sarthak’s Bot: Namaste! kun samaya ma alarm set garna chahanu huncha? (jaste: malai 7 30 ma set gardinu)")

    hour = None
    minute = None
    period = None

    while True:
        user_input = input("User: ").strip()
        if not user_input:
            continue

        known = {"hour": hour, "minute": minute, "period": period}
        extracted = extract_alarm_time(client, model_name, user_input, known)

        if extracted.get("hour") is not None:
            hour = int(extracted["hour"])
        if extracted.get("minute") is not None:
            minute = int(extracted["minute"])
        if extracted.get("period") in ["AM", "PM"]:
            period = extracted["period"]

        # Prompt for missing pieces naturally
        if hour is None:
            print("Sarthak’s Bot: kun samaya ma (1–12) bhitra rojnus")
            continue
        if minute is None:
            minute = 0
        if period is None:
            print(f"Sarthak’s Bot: thik cha— ‘{hour}:{minute:02d}’ bhayo. bihana ho ki beluka? (AM/PM)")
            continue

        try:
            if not (1 <= hour <= 12 and 0 <= minute <= 59):
                raise ValueError

            conv_hour = hour % 12 + (12 if period == 'PM' else 0)
            now = datetime.now()
            alarm_time = now.replace(hour=conv_hour, minute=minute, second=0, microsecond=0)
            if alarm_time <= now:
                alarm_time += timedelta(days=1)

            print(f"Sarthak’s Bot: thikai cha! maile {alarm_time.strftime('%I:%M %p')} ma alarm set gare 🙂")
            time.sleep((alarm_time - now).total_seconds())
            print("⏰ Sarthak’s Bot: samaya bhayo! alarm bajirahi cha.")
            return alarm_time

        except ValueError:
            print("Sarthak’s Bot: ma samaya bujhina, feri prayas garaun.")
            hour = minute = period = None


if __name__ == "__main__":
    alarm_time = chatbot_alarm()
    print(f"Sarthak’s Bot: alarm schedule gareko samaya: {alarm_time}")


Sarthak’s Bot: Namaste! kun samaya ma alarm set garna chahanu huncha? (jaste: malai 7 30 ma set gardinu)
AI Json response Backend ```json
{"hour":null,"minute":null,"period":"AM"}
```
Sarthak’s Bot: kun samaya ma (1–12) bhitra rojnus
AI Json response Backend ```json
{"hour":9,"minute":0,"period":"AM"}
```
Sarthak’s Bot: thikai cha! maile 09:00 AM ma alarm set gare 🙂
